In [1]:
import whisper
import os
import ffmpeg


def extract_audio(mp4_file, audio_file):
    ffmpeg.input(mp4_file).output(audio_file).run()

In [2]:
mp4_name = "exercise.mp4"

path = r"../../Downloads/huberman.mp4"
print(os.path.isfile(path))
print(path)
extract_audio(path, "audio1.mp3")

True
../../Downloads/huberman.mp4


In [3]:
model = whisper.load_model("base")  

result = model.transcribe("audios/audio1.mp3", word_timestamps=True, verbose = True)

Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English


C:\Users\Jan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
C:\Users\Jan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


[00:03.740 --> 00:08.560]  I wake up in the morning and I want to reach for my phone, but I know that even if I were
[00:08.560 --> 00:13.700]  to crank up the brightness on that phone screen, it's not bright enough to trigger that cortisol
[00:13.700 --> 00:19.500]  spike and for me to be at my most alert and focus throughout the day and to optimize my
[00:19.500 --> 00:20.040]  sleep at night.
[00:20.200 --> 00:23.640]  So what I do is I get out of bed and I go outside.


C:\Users\Jan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
C:\Users\Jan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


[00:23.640 --> 00:30.700]  And if it's a bright clear day and the sun is low in the sky or the sun is starting to
[00:30.700 --> 00:35.220]  get overhead what we call low solar angle, then I know I'm getting outside at the right
[00:35.220 --> 00:35.560]  time.
[00:35.700 --> 00:40.560]  If there's cloud cover and I can't see the sun, I also know I'm doing a good thing because
[00:40.560 --> 00:44.240]  it turns out especially on cloudy days you want to get outside and get as much light
[00:44.240 --> 00:45.740]  energy or photons in your eyes.
[00:46.340 --> 00:49.940]  But let's say it's a very clear day and I can see where the sun is.
[00:49.940 --> 00:52.880]  I do not need to stare directly into the sun.
[00:53.240 --> 00:57.660]  If it's very low in the sky, I might do that because it's not going to be very painful
[00:57.660 --> 00:58.460]  to my eyes.
[00:58.540 --> 01:02.840]  However, if the sun is a little bit brighter and a little bit higher in the sky, sometimes
[01:02.840

In [4]:
def format_timestamp(
    seconds: float, always_include_hours: bool = False, decimal_marker: str = "."
):
    assert seconds >= 0, "non-negative timestamp expected"
    milliseconds = round(seconds * 1000.0)

    hours = milliseconds // 3_600_000
    milliseconds -= hours * 3_600_000

    minutes = milliseconds // 60_000
    milliseconds -= minutes * 60_000

    seconds = milliseconds // 1_000
    milliseconds -= seconds * 1_000

    hours_marker = f"{hours:02d}:" if always_include_hours or hours > 0 else ""
    return (
        f"{hours_marker}{minutes:02d}:{seconds:02d}{decimal_marker}{milliseconds:03d}"
    )

In [5]:
print(result.keys())

    
tekst = ""
for segment in result['segments']:
    start, end, text = segment["start"], segment["end"], segment["text"]
    line = f"[{format_timestamp(start)} --> {format_timestamp(end)}] {text}\n"
    tekst += line

dict_keys(['text', 'segments', 'language'])


In [6]:
print(tekst)

[00:03.740 --> 00:08.560]  I wake up in the morning and I want to reach for my phone, but I know that even if I were
[00:08.560 --> 00:13.700]  to crank up the brightness on that phone screen, it's not bright enough to trigger that cortisol
[00:13.700 --> 00:19.500]  spike and for me to be at my most alert and focus throughout the day and to optimize my
[00:19.500 --> 00:20.040]  sleep at night.
[00:20.200 --> 00:23.640]  So what I do is I get out of bed and I go outside.
[00:23.640 --> 00:30.700]  And if it's a bright clear day and the sun is low in the sky or the sun is starting to
[00:30.700 --> 00:35.220]  get overhead what we call low solar angle, then I know I'm getting outside at the right
[00:35.220 --> 00:35.560]  time.
[00:35.700 --> 00:40.560]  If there's cloud cover and I can't see the sun, I also know I'm doing a good thing because
[00:40.560 --> 00:44.240]  it turns out especially on cloudy days you want to get outside and get as much light
[00:44.240 --> 00:45.740]  ener

In [7]:
def find_terminal(text):
    #print(text)
    num = 10000
    for char in [".", "?", "!"]:
        temp = text.find(char)
        num = temp if temp < num and temp != -1 else num 
    return num

def find_nth(haystack: str, needle: str, n: int) -> int:
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

def find_where_split(tekst:str, num_of_words:int = 1000):
    lista = []
    num_blanks = tekst.count(' ')
    iteration = 1

    if num_blanks<num_of_words:
        return None
    
    while True:
        spl = find_nth(tekst, ' ',num_of_words*iteration)
        #print(find_terminal(tekst[spl:]))
        spl += find_terminal(tekst[spl:])
        lista.append(spl)

        iteration += 1

        if iteration*num_of_words > num_blanks:
            break
    
    lista.append(len(tekst)-1)
    return lista


def split_into_pieces(tekst:str, num_of_words:int = 1000) -> list:
    lista = []

    where_split = find_where_split(tekst, num_of_words)
    temp = 0
    for idx in where_split:
        lista.append(tekst[temp:idx+1])
        temp = idx+1

    return lista

In [12]:
lista = split_into_pieces(result['text'])
for idx, i in enumerate(lista):
    print(i)
    lista[idx] += '\n'

f = open("batched_transcript/transcript.txt", "w")
f.writelines(lista)
f.close()

 I wake up in the morning and I want to reach for my phone, but I know that even if I were to crank up the brightness on that phone screen, it's not bright enough to trigger that cortisol spike and for me to be at my most alert and focus throughout the day and to optimize my sleep at night. So what I do is I get out of bed and I go outside. And if it's a bright clear day and the sun is low in the sky or the sun is starting to get overhead what we call low solar angle, then I know I'm getting outside at the right time. If there's cloud cover and I can't see the sun, I also know I'm doing a good thing because it turns out especially on cloudy days you want to get outside and get as much light energy or photons in your eyes. But let's say it's a very clear day and I can see where the sun is. I do not need to stare directly into the sun. If it's very low in the sky, I might do that because it's not going to be very painful to my eyes. However, if the sun is a little bit brighter and a litt

In [ ]:
# num = 0
# for word in result['segments']:
#     for w in word['words']:
#         num += 1
# print(num)

# t = ""
# flag = False
# for word in result['segments']:
#     for w in word['words']:
#         t += w['word']

#     if x>:
# print(t)


In [13]:
f = open("transcripts/transcript.txt", "w")
f.write(result['text'])
f.close()

In [33]:
#tekst = tekst.split('\n')

f = open('final_fragments/fragments1.txt', encoding='utf-8')
fragments = f.read().split('\n')
fragments.remove('')
print(fragments)
print(tekst)

lista = []

for i in fragments:
    temp = []
    flag = False
    for idx, el in enumerate(tekst):
        if i[:10] in el and not flag:
            temp.append(idx)
            flag = True

        if i[-5:] in el and flag:
            temp.append(idx)
            flag = False
            
    lista.append(temp)
print(lista)

for i in lista:
    for num in range(i[0], i[1]+1):
        print(tekst[num])
    
    print("-----------------")


["I wake up in the morning and I want to reach for my phone, but I know that even if I were to crank up the brightness on that phone screen, it's not bright enough to trigger that cortisol spike and for me to be at my most alert and focus throughout the day and to optimize my sleep at night.", "So trying it outside, ideally within the first five minutes of waking or maybe it's 15 minutes, but certainly within the first hour after waking. I want to share with you three critical things about this tool of morning sunlight viewing. First of all, this is not some woo biology thing. This is grounded in the core of our physiology. There are literally hundreds, if not thousands of quality peer-reviewed papers showing that light viewing early in the day is the most powerful stimulus for wakefulness throughout the day and it has a powerful positive impact on your ability to fall and stay asleep at night."]
['[00:03.740 --> 00:08.560]  I wake up in the morning and I want to reach for my phone, bu